# Let's Make the Call to compute_trunc_likes not so slow

## Setup

In [1]:
import numpy as np
from scipy import special
import math

In [2]:
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(np.divide(x - mu, tau), 2))
    return log_likelihood

In [16]:
tau = .05
bounds = np.array([[-1,  1],
       [ 0,  1],
       [ 0,  1],
       [-1,  1],
       [-1,  1],
       [-1,  1]])
which_param = 0

nth_grp_lvl_param = np.load('../nth_grp_lvl_param.npy')
nth_prev_iter_curve_param = np.load('../nth_prev_iter_curve_param.npy')

## Original Call

In [4]:
test = np.array([compute_trunc_likes(nth_grp_lvl_param[:, i], nth_prev_iter_curve_param[i])
                                    for i in range(len(nth_prev_iter_curve_param))]).T
test

array([[ -79.48709185, -150.99830714,  -29.47795205, ...,  -71.2268334 ,
        -105.3245468 ,  -54.88876175],
       [-121.67798878, -207.17952733,  -57.37657632, ..., -111.4520988 ,
        -153.09262719,  -27.67273552],
       [ -72.56912216, -141.4638131 ,  -25.23297847, ...,  -64.67648373,
         -97.3635107 ,  -60.95154324],
       ...,
       [-472.7931511 , -629.58835366, -335.65936243, ..., -452.55054658,
        -532.62870847,  -25.09529379],
       [-153.45980969, -247.96357878,  -79.96187393, ..., -141.96911203,
        -188.4631494 ,  -14.79389256],
       [-203.46263452, -310.43270377, -117.22934177, ..., -190.22042961,
        -243.4356123 ,   -2.95057406]])

In [5]:
np.shape(test)

(100000, 20)

In [6]:
test[0]

array([ -79.48709185, -150.99830714,  -29.47795205,    1.72411827,
        -58.22374008,   -1.62280954,  -52.41845844,   -2.82212032,
        -23.82987544, -115.64009915, -190.95174464,  -76.72652552,
        -51.11403342,  -69.8290162 ,  -97.69304995,   -9.24178285,
         -0.26000773,  -71.2268334 , -105.3245468 ,  -54.88876175])

In [7]:
%%timeit
test = np.array([compute_trunc_likes(nth_grp_lvl_param[:, i], nth_prev_iter_curve_param[i])
                                    for i in range(len(nth_prev_iter_curve_param))]).T

91.6 ms ± 8.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Vectorize?
Instead of acting over a single mu or column, I need operations that use the entire array such that `mu` is a vector and `x` is a 2-dimensional array.

In [ ]:
mu = nth_prev_iter_curve_param
x = nth_grp_lvl_param

log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
    -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
        -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
        -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(np.divide(x - mu, tau), 2))

log_likelihood

In [ ]:
np.shape(log_likelihood)

In [ ]:
log_likelihood

In [ ]:
log_likelihood[0]

In [8]:
%%timeit
mu = nth_prev_iter_curve_param
x = nth_grp_lvl_param

log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
    -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
        -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
        -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(np.divide(x - mu, tau), 2))

85.1 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


It doesn't seem the code needs further alterations to support vectorization.

## Result
Let's test a call to compute_trunc_likes using the arrays.

In [ ]:
result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
result

In [ ]:
np.shape(result)

In [9]:
%%timeit
result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

85.1 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## This Needs to Be Several Times Faster. Where is the Bottleneck?

In [ ]:
%%timeit

# get rid of first term
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(np.divide(x - mu, tau), 2))
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [ ]:
%%timeit

# then the second hidden in the log term
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log((np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(np.divide(x - mu, tau), 2))
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [ ]:
%%timeit

# the entire second log term
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) ) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(np.divide(x - mu, tau), 2))
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [ ]:
%%timeit
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2))))))))- np.multiply(.5, np.power(np.divide(x - mu, tau), 2))
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [ ]:
%%timeit

# the final term, the only one using x
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi))
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

It's the last term. I need to focus all efforts on optimizing the last term. Any  particular part?

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.power(np.divide(x - mu, tau), 2)
    return log_likelihood


result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x - mu, tau))
    return log_likelihood


result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(x - mu, 2))
    return log_likelihood


result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(np.divide(x, tau), 2))
    return log_likelihood


result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

## The Bottleneck is np.power
It's about 82ms with the full equation. The full term is `np.multiply(.5, np.power(np.divide(x - mu, tau), 2))`. Without it, speed is 19.8us. The contribution of each part?

Removing np.multiply obtains a runtime of 78.5ms.
Removing np.power obtains 22.1ms.
Removing divide gets 79.7ms.
Removing x-mu gets 77.6.

So the most impactful statement is np.power. Can I make it faster?

In [10]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.power(np.divide(x - mu, tau), 2))
    return log_likelihood


result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

87.7 ms ± 3.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x - mu, tau) ** 2)
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

29 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


That's nice but I still need it to be 4 times faster to be MATLAB tier. 

## What's the Top Bottleneck Now?
The full term is now `np.multiply(.5, np.divide(x - mu, tau) ** 2)`. Takes 26.2ms to run. I need it down to 6.5ms. 

In [12]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x - mu, tau) ** 2)
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

29.2 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Lesioning

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi))
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

remove multiply

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.divide(x - mu, tau) ** 2)
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

remove divide

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, (x - mu) ** 2)
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

remove sum

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x, tau) ** 2)
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

remove power

In [ ]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x - mu, tau))
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

Runtime is similar across all operations now. While removing them all obtains huge gains, removing any individual operation only gives about 5ms. 

In [13]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (x - mu)
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

13 ms ± 57.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Even if it were just an addition operation, it'd take 12ms instead of my 6.5.

## The Number Precision Experiment
Let's take a step back and see if PCIT does ok with lower number precision.

In [14]:
nth_grp_lvl_param = nth_grp_lvl_param.astype('float32')
nth_prev_iter_curve_param = nth_prev_iter_curve_param.astype('float32')

In [15]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x - mu, tau) ** 2)
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

14.7 ms ± 330 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Half precision grants half-speed runtime. Does PCIT work fine that way?

```
Start time 12/17 9:53
********** START OF MESSAGES **********
0 trials are dropped since they are regarded as outliers
********** END OF MESSAGES **********
Betas: 0, 1
EM Iteration: 0
Optimization terminated successfully.
         Current function value: 1243.911318
         Iterations: 6
         Function evaluations: 8
         Gradient evaluations: 8
Betas: 0.10592798970016724, 0.6316629314995463
EM Iteration: 1
```

Not too far off from Betas: 0.0882896038554186, 0.6902758140586037. But definitely meaningfully off. And the extra benefit isn't double. I found an improve from 4x slower to just 3x slower. So I'm ambivalent about the idea. But at least I've figured out how to institute it.

## Can We Vectorize The Loop Over Params?
To vectorize, I have to use every row of `param` at once.
`nth_grp_lvl_param` is just a selected column of param, repeated for 20 columns. Could I avoid doing that?
`idx` selects a subset of rows of `prev_iter_curve_param` to fill `nth_prev_iter_curve_param` while `npm` similarly picks the column filling the variable.

`nth_grp_lvl_param` has shape 100,000x20, while `prev_iter_curve_param` has shape 20. Each entry in `prev_ter_curve_param` is broadcast to each column in `nth_grp_lvl_param` during all this math - well, after a bit of computation on those entries.

In [17]:
nParam = 6
tau = .05
bounds = np.array([[-1,  1],
       [ 0,  1],
       [ 0,  1],
       [-1,  1],
       [-1,  1],
       [-1,  1]])
which_param = 0
idx = 0
reduced_nParticles = 20

nth_grp_lvl_param = np.load('../nth_grp_lvl_param.npy')
nth_prev_iter_curve_param = np.load('../nth_prev_iter_curve_param.npy')
nth_prev_iter_curve_param

array([-0.6581, -0.8944, -0.4167,  0.0225,  0.5296, -0.1555,  0.5025,
       -0.176 , -0.3794,  0.7477,  0.9636, -0.6472, -0.5352, -0.6191,
        0.6868,  0.2184,  0.0886, -0.6249, -0.7523,  0.5142])

In [12]:
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x - mu, tau) ** 2)
    return log_likelihood

result = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

In [55]:
alt_nth_grp_lvl_param = nth_grp_lvl_param
alt_nth_prev_iter_curve_param = np.tile(nth_prev_iter_curve_param, (6,1))

In [61]:
compute_trunc_likes(alt_nth_grp_lvl_param, alt_nth_prev_iter_curve_param.T)

ValueError: operands could not be broadcast together with shapes (100000,20) (20,6) 

In [59]:
np.shape(alt_nth_prev_iter_curve_param)

(6, 20)

In [57]:
nth_prev_iter_curve_param

array([-0.6581, -0.8944, -0.4167,  0.0225,  0.5296, -0.1555,  0.5025,
       -0.176 , -0.3794,  0.7477,  0.9636, -0.6472, -0.5352, -0.6191,
        0.6868,  0.2184,  0.0886, -0.6249, -0.7523,  0.5142])

In [62]:
np.shape(compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param))

(100000, 20)

So I can get the same result using just a single column of `nth_grp_lvl_param` and a transpose. Could that improve runtime? How do I find out? Compare runtime against using the full number. It actu

In [ ]:
for npm in range(nParam):
    which_param = npm
    nth_grp_lvl_param = np.tile(
        param[:, npm].reshape(-1, 1), (1, reduced_nParticles))
    nth_prev_iter_curve_param = prev_iter_curve_param[target_indices, npm]
    trunc_likes = compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)
    prob_grp_lvl_curve = np.add(prob_grp_lvl_curve, trunc_likes)

    if np.any(np.isnan(prob_grp_lvl_curve)):
        raise ValueError('NaNs in probability of group level curves matrix!')

## Can I Optimize The Other Operations?

In [142]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x - mu, tau) ** 2)
    return log_likelihood

compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

27 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [99]:
#%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (.5 * (((x - mu)/ tau) ** 2))
    return log_likelihood

compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

array([[ -79.48709185, -150.99830714,  -29.47795205, ...,  -71.2268334 ,
        -105.3245468 ,  -54.88876175],
       [-121.67798878, -207.17952733,  -57.37657632, ..., -111.4520988 ,
        -153.09262719,  -27.67273552],
       [ -72.56912216, -141.4638131 ,  -25.23297847, ...,  -64.67648373,
         -97.3635107 ,  -60.95154324],
       ...,
       [-472.7931511 , -629.58835366, -335.65936243, ..., -452.55054658,
        -532.62870847,  -25.09529379],
       [-153.45980969, -247.96357878,  -79.96187393, ..., -141.96911203,
        -188.4631494 ,  -14.79389256],
       [-203.46263452, -310.43270377, -117.22934177, ..., -190.22042961,
        -243.4356123 ,   -2.95057406]])

## Focus On Just The Time-Consuming Term

In [136]:
%%timeit
def quickie(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')
        
    return np.multiply(.5, np.divide(x - mu, tau) ** 2)

quickie(nth_grp_lvl_param, nth_prev_iter_curve_param)

20.9 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [141]:
%%timeit
def quickie(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')
        
    return (((x - mu)/ constant) ** 2)

quickie(nth_grp_lvl_param, nth_prev_iter_curve_param)

15.9 ms ± 261 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [132]:
constant = tau/np.sqrt(.5)

So I can stick the `.5` multiplication term onto `tau`, so that all the arithmetic broadcasting only happens once.

## Collapse The Arithmetic
So I can stick the `.5` multiplication term onto `tau`, so that all the arithmetic broadcasting only happens once.

In [20]:
tau = .05
bounds = np.array([[-1,  1],
       [ 0,  1],
       [ 0,  1],
       [-1,  1],
       [-1,  1],
       [-1,  1]])
which_param = 0

nth_grp_lvl_param = np.load('../nth_grp_lvl_param.npy')
nth_prev_iter_curve_param = np.load('../nth_prev_iter_curve_param.npy')

In [21]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - np.multiply(.5, np.divide(x - mu, tau) ** 2)
    return log_likelihood

compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

26.2 ms ± 1.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
nth_grp_lvl_param = nth_grp_lvl_param.astype('float32')
nth_prev_iter_curve_param = nth_prev_iter_curve_param.astype('float32')

In [23]:
%%timeit 
def compute_trunc_likes(x, mu):
    global tau, bounds, which_param

    if tau <= 0:
        raise ValueError('Tau is <= 0!')

    # This ugly thing below is a manifestation of log(1 ./ (tau .* (normcdf((bounds(which_param, 2) - mu) ./ tau) -
    # normcdf((bounds(which_param, 1) - mu) ./ tau))) .* normpdf((x - mu) ./ tau)) Refer to
    # http://en.wikipedia.org/wiki/Truncated_normal_distribution for the truncated normal distribution
    log_likelihood = -(np.log(tau) + np.log(np.multiply(0.5, special.erfc(
        -np.divide(bounds[which_param, 1] - mu, np.multiply(tau, math.sqrt(2))))) + (np.multiply(-0.5, special.erfc(
            -np.divide(bounds[which_param, 0] - mu, np.multiply(tau, math.sqrt(2)))))))) + np.multiply(
            -.5, np.log(2) + np.log(np.pi)) - (np.divide(x - mu, tau/np.sqrt(.5)) ** 2)
    return log_likelihood

compute_trunc_likes(nth_grp_lvl_param, nth_prev_iter_curve_param)

10.5 ms ± 173 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
